<p style="font-family: Arial; font-size:3em;color:black;"> Lab Exercise 10</p>

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(5)  # creating pseudo-random numbers for reproducibility

In [25]:
# Let's consider the following A_orig matrix:

A_orig = np.array([[4, 2, 4, 5, 4, 2, 2], [4, 4, 3, 2, 2, 5, 4], [4, 1, 4, 7, 3, 6, 2], [8 ,1, 2, 0, 5, 0, 7], [4 , 5, 8, 7, 6, 2, 3]],dtype=float)
M, N = 5, 7
print (pd.DataFrame(A_orig).head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  1.0  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  3.0


In [26]:
# Let's add some NaN(s) to  A_orig matrix

A = A_orig.copy()
A[3][1] = np.NaN
A[4][6] = np.NaN

A_df = pd.DataFrame(A)
print (A_df.head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  NaN  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  NaN


In [27]:
def compute_missing_error(original_matrix, reconstructed_matrix, missing_indices):
    errors = []
    for (i, j) in missing_indices:
        original_value = original_matrix[i, j]
        predicted_value = reconstructed_matrix[i, j]
        error = abs(original_value - predicted_value) / original_value * 100
        errors.append(error)
    return np.mean(errors)


In [28]:
# Fine tune K value such that the (average percentage) error for the 2 missing elements is minimum.
# Remember: this is an exercise where we have the actual values for missing elements. 
# In real-life scenarios missins elements are not known and you may need different metric(s)
# to assess the quality of your reconstructed matrix.
K = 1
P = np.abs(np.random.uniform(low=0, high=8, size=(M, K)))
Q = np.abs(np.random.uniform(low=0, high=8, size=(K, N)))
P = np.divide(P, K*P.max())
Q = np.divide(Q, K*Q.max())

In [31]:
def matrix_factorization(Rating_Matrix, P, Q, K, steps, alpha=0.001, beta=0.02):
    for step in range(steps):
        for i in range(len(Rating_Matrix)):  
            for j in range(len(Rating_Matrix[i])):  
                if ~np.isnan(Rating_Matrix[i][j]):  
                    
                    eij = Rating_Matrix[i][j] - np.dot(P[i, :], Q[:, j])
                    for k in range(K):
                        P[i][k] += alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] += alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        #computing total error
        e = 0
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    e += pow(Rating_Matrix[i][j] - np.dot(P[i, :], Q[:, j]), 2)
                    for k in range(K):
                        e += (beta / 2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        if step % 100 == 0:
            print(f"Step {step}, Total Error: {e:.4f}")
        if e < 0.0001:
            break
    return P, Q


In [34]:

missing_indices = [(3, 1), (4, 6)]  

for K in range(2, 10):
    P = np.abs(np.random.uniform(low=0, high=8, size=(M, K)))
    Q = np.abs(np.random.uniform(low=0, high=8, size=(K, N)))
    P = np.divide(P, K * P.max())
    Q = np.divide(Q, K * Q.max())

    print(f"Testing K={K}: P shape={P.shape}, Q shape={Q.shape}")
    
    
    eP, eQ = matrix_factorization(A, P, Q, K, steps=1000)
    
    
    eA = np.matmul(eP, eQ)
    
    
    error = compute_missing_error(A_orig, eA, missing_indices)
    print(f"K={K}, Average Percentage Error={error:.2f}%")


Testing K=2: P shape=(5, 2), Q shape=(2, 7)
Step 0, Total Error: 608.9646
Step 100, Total Error: 106.9289
Step 200, Total Error: 49.7035
Step 300, Total Error: 41.1122
Step 400, Total Error: 39.6085
Step 500, Total Error: 38.6657
Step 600, Total Error: 38.1036
Step 700, Total Error: 37.7735
Step 800, Total Error: 37.5756
Step 900, Total Error: 37.4534
K=2, Average Percentage Error=457.98%
Testing K=3: P shape=(5, 3), Q shape=(3, 7)
Step 0, Total Error: 611.4907
Step 100, Total Error: 105.6290
Step 200, Total Error: 48.7233
Step 300, Total Error: 35.7977
Step 400, Total Error: 26.3224
Step 500, Total Error: 18.9023
Step 600, Total Error: 14.8089
Step 700, Total Error: 12.5641
Step 800, Total Error: 11.4215
Step 900, Total Error: 10.8613
K=3, Average Percentage Error=409.25%
Testing K=4: P shape=(5, 4), Q shape=(4, 7)
Step 0, Total Error: 623.8044
Step 100, Total Error: 116.0555
Step 200, Total Error: 97.5362
Step 300, Total Error: 46.6994
Step 400, Total Error: 31.4878
Step 500, Total E

In [ ]:
# for me k=7 has the average error of 242.25% which is pretty high but for this case this one is the best option.
# I can normalize and scale the A_org matrix to get better value or increase the number of iterations to help model improve over time.